Project 2: Model Engineering
===

___

Submitted by:

* Arthur Humblot
* Bekhzod Anvarov
* Ghita El Belghiti


University: **Politechnico di Torino**

Academic Year: **2025 - 2026**

## 1. Task 1: Frequency-based baseline

## 2. Task 2: Feed Forward Neural Network (FFNN)

## 3. Task 3: Recurrent Neural Network (RNN)

## 4. Task 1: Graph Neural Network (GNN)